<a href="https://colab.research.google.com/github/Que1Pereza2/Mr.CrabsAnalyzer/blob/main/CanYouFeelItNow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports block

In [11]:
import numpy as np
import pandas as pd
import re
import nltk
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

This block reads the None2775.csv file and creates the arrays features and labels.

In [10]:
reviews = pd.read_csv("None2775.csv")

features = reviews.iloc[:,0].values
labels = reviews.iloc[:,1].values

This function uses regex to clean the data and prepares it for the neural network to train on.

In [12]:
processed_features = []
for sentence in range(0, len(features)):
    # Remove all the special characters
    processed_feature = re.sub(r'\W', ' ', str(features[sentence]))

    # remove all single characters
    processed_feature= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)

    # Remove single characters from the start
    processed_feature = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_feature)

    # Substituting multiple spaces with single space
    processed_feature = re.sub(r'\s+', ' ', processed_feature, flags=re.I)

    # Removing prefixed 'b'
    processed_feature = re.sub(r'^b\s+', '', processed_feature)

    # Converting to Lowercase
    processed_feature = processed_feature.lower()

    processed_features.append(processed_feature)


Creating and feeding the data to the Neural Network

In [13]:
vectorizer = TfidfVectorizer (max_features=2500, min_df=7, max_df=0.8)
processed_features = vectorizer.fit_transform(processed_features).toarray()

X_train, X_test, y_train, y_test = train_test_split(processed_features, labels, test_size=0.2, random_state=0)

text_classifier = RandomForestClassifier(n_estimators=200, random_state=0)
text_classifier.fit(X_train, y_train)

predictions = text_classifier.predict(X_test)



Neural Network stats

In [9]:
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))

[[ 237  308]
 [  46 1420]]
              precision    recall  f1-score   support

         "0"       0.84      0.43      0.57       545
         "1"       0.82      0.97      0.89      1466

    accuracy                           0.82      2011
   macro avg       0.83      0.70      0.73      2011
weighted avg       0.83      0.82      0.80      2011

0.8239681750372949
